# Summary

Explore if the chosen types are correct and inspect some values in the full table.

# Initialize

In [1]:
import logging

import pandas as pd
import psycopg2

In [2]:
conn = psycopg2.connect(database='etldb',
                        user='airflow_etl',
                        host='localhost',
                        port=5432,
                        options=f'-c search_path=etl')
conn.autocommit = True


def run_query(sql, conn=conn, **read_sql_kwargs) -> pd.DataFrame:
    return pd.read_sql(sql, con=conn, **read_sql_kwargs)


# Set logging
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()

# Check range of values

In [3]:
integer_cols = [
    'cicid',
    'i94yr',
    'i94mon',
    'i94cit',
    'i94res',
    'arrdate',
    'i94mode',
    'depdate',
    'i94bir',
    'i94visa',
    'count',
    'biryear',
    'admnum',
]

text_cols = [
    'i94port',
    'i94addr',
    'dtadfile',
    'visapost',
    'occup',
    'entdepa',
    'entdepd',
    'entdepu',
    'matflag',
    'dtaddto',
    'gender',
    'insnum',
    'airline',
    'fltno',
    'visatype',
]

In [4]:
one_row = run_query('SELECT * FROM immigration LIMIT 1;')
cols = one_row.columns.tolist()

In [5]:
one_row.T

,0
cicid,7
i94yr,2016
i94mon,1
i94cit,101
i94res,101
i94port,BOS
arrdate,20465
i94mode,1
i94addr,MA
depdate,None


# Numerical types storage size

In [ ]:
# ~4 min to run this cell

col_types_integer = []

logging.info('Getting the maximum and minimum values of numeric columns...')
for col in integer_cols:
    logging.info(f'Column "{col:s}"')
    result = run_query(f'''
    WITH
    min_max AS (
      SELECT MIN({col:s}) AS min_{col:s}, MAX({col:s}) AS max_{col:s}
      FROM immigration)
    
    SELECT
      CASE
        WHEN min_{col:s} >= -32768 AND max_{col:s} <= 32767 THEN 'SMALLINT'
        WHEN min_{col:s} >= -2147483648 AND max_{col:s} <= 2147483647 THEN 'INTEGER'
        WHEN min_{col:s} >= -9223372036854775808 AND max_{col:s} <= 9223372036854775807 THEN 'BIGINT'
        ELSE 'UNKNOWN'
      END AS numeric_type,
      min_{col:s} AS min_value, max_{col:s} AS max_value
    FROM min_max;''').loc[0].to_dict()
    col_types_integer.append(dict({'col_name': col}, **result))
    logging.info('Done!')

In [7]:
# Checking if the chosen numerical types in the CREATE TABLE make sense.

pd.DataFrame(col_types_integer)

,col_name,numeric_type,min_value,max_value
0,cicid,INTEGER,1,7667577
1,i94yr,SMALLINT,2016,2016
2,i94mon,SMALLINT,1,12
3,i94cit,SMALLINT,0,999
4,i94res,SMALLINT,101,760
5,arrdate,SMALLINT,20454,20819
6,i94mode,SMALLINT,0,9
7,depdate,INTEGER,-14388,48342
8,i94bir,SMALLINT,-3,1812
9,i94visa,SMALLINT,1,3


In [8]:
# Total number of rows for comparison: ~ 40 million rows

run_query('''SELECT COUNT(*)
FROM immigration;''')

,count
0,40790529


In [9]:
# Checking the negative values in "i94bir"
# Only 10 values

run_query('''SELECT COUNT(*)
FROM immigration
WHERE i94bir < 0;''')

,count
0,10


In [10]:
# Looking at the table, the rest of the data seems OK, except the "biryear"
# They happen in 2 airports ("AGA" and "SAI"). We can discard these rows.

run_query('''SELECT i94bir, *
FROM immigration
WHERE i94bir < 0;''')

,i94bir,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,-1,661795,2016,12,209,209,AGA,20792,1,NV,...,None,M,2017,01162017,F,3666,JE,41850331528,00941,GMT
1,-2,5945548,2016,1,252,209,AGA,20455,1,GU,...,None,None,2018,02152016,F,39552,UA,37546316627,00178,GMT
2,-2,6491820,2016,3,254,276,SAI,20517,1,None,...,None,None,2018,04172016,U,3980,OZ,46136965533,00625,GMT
3,-3,5952559,2016,4,252,209,AGA,20554,1,None,...,None,None,2019,05242016,M,None,None,57545310233,00001,GMT
4,-1,1718396,2016,11,254,276,AGA,20768,1,GU,...,None,None,2017,12252016,F,3681,KE,24807322027,02115,GMT
5,-1,1608057,2016,12,254,276,AGA,20797,1,GU,...,None,None,2017,01222017,F,3664,KE,42070617528,00113,GMT
6,-1,2213228,2016,12,254,276,SAI,20800,1,KY,...,None,None,2017,01252017,F,3980,OZ,46240307833,00627,GMT
7,-1,1608613,2016,12,254,276,AGA,20797,1,GU,...,None,None,2017,01222017,F,3696,KE,42038107028,00113,GMT
8,-1,2415887,2016,12,254,276,AGA,20801,1,None,...,None,None,2017,01262017,M,3938,KE,42091273728,00113,GMT
9,-1,7129291,2016,12,209,209,AGA,20819,1,GU,...,None,None,2017,02142017,F,3703,JL,41840885228,00941,GMT


In [11]:
# Checking the negative values in "depdate"
# Only 52 rows

run_query('''SELECT COUNT(*)
FROM immigration
WHERE depdate < 0;''')

,count
0,52


In [12]:
# Looking at the table, the rest of the data seems OK.

run_query('''SELECT depdate, *
FROM immigration
WHERE depdate < 0
LIMIT 10;''')

,depdate,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,-14011,7010885,2016,7,255,255,HIG,20664,3,MA,...,None,M,1953,01282017,M,4927,None,3946338085,None,B2
1,-14388,6881939,2016,7,108,108,PEM,20664,3,NC,...,None,M,1976,10262016,F,None,None,3812096985,LAND,WT
2,-14021,2092318,2016,8,131,131,XXX,20675,3,AK,...,None,M,1968,11062016,M,None,None,17107227727,LAND,WT
3,-14359,3604434,2016,8,148,112,NIA,20682,3,NY,...,None,M,1975,11132016,F,None,None,5591306585,LAND,WT
4,-6717,2525843,2016,8,117,117,THO,20677,3,PA,...,None,M,2005,11082016,F,None,None,64822333733,LAND,WT
5,-14342,4848615,2016,9,135,135,SWE,20719,3,MT,...,None,M,1989,12202016,M,None,None,17739618627,LAND,WT
6,-13977,5178618,2016,9,111,111,DER,20720,3,ME,...,None,M,1996,12212016,F,19182,None,11995762285,LAND,WT
7,-14375,5173886,2016,8,253,253,DET,20688,3,MI,...,None,M,2002,02212017,F,None,None,6769697085,LAND,B2
8,-14307,4329895,2016,9,130,130,BLA,20716,3,WA,...,None,M,1989,12172016,F,None,None,11333437685,LAND,WT
9,-14307,4359043,2016,9,130,130,BLA,20716,3,WA,...,None,M,1989,12172016,M,None,None,11333906085,LAND,WT


# Text columns length

In [ ]:
# ~6 min to run this cell

col_types_text = []

logging.info('Getting the maximum and minimum lengths of text columns...')
for col in text_cols:
    logging.info(f'Column "{col:s}"')
    result = run_query(f'''
      SELECT MIN(LENGTH({col:s})) AS min_length, MAX(LENGTH({col:s})) AS max_length
      FROM immigration;''').loc[0].to_dict()
    col_types_text.append(dict({'col_name': col}, **result))
    logging.info('Done!')

In [14]:
# The text types in the CREATE TABLE statement were decided here:
# If min = max --> CHAR(min)
# Otherwise --> VARCHAR(max)
# Exceptions: visapost, occup, insnum, airline, fltno, visatype: VARCHAR(16) to
# leave room for more characters, since this is a sample.

pd.DataFrame(col_types_text)

,col_name,min_length,max_length
0,i94port,3,3
1,i94addr,1,2
2,dtadfile,8,8
3,visapost,3,3
4,occup,3,3
5,entdepa,1,1
6,entdepd,1,1
7,entdepu,1,1
8,matflag,1,1
9,dtaddto,2,8


# Inspect values

In [15]:
# run_query('SELECT ... FROM immigration;')

# Cleanup

In [16]:
conn.close()